In [3]:
restart = True
epoch_to_pickup = 0

In [4]:
# Import libraries

from tensorflow.keras.layers.experimental.preprocessing import StringLookup
import numpy as np
import os
import time
import random
import contextlib
import io
import re
import string
import gc  # Import the garbage collector module
import unicodedata
from plyer import notification

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [5]:
# Get a list of GPU devices
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        # Enable memory growth for all GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # After enabling memory growth, we need to make sure TensorFlow sees the updated configuration
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(f"Error setting memory growth: {e}")

1 Physical GPUs, 1 Logical GPUs


In [6]:
path = ''

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')
# path = '/content/drive/My Drive/M6_Fall2023e/'

## Functions for downloading text


In [8]:
def preprocess_text(text):

    text = text.replace("Project Gutenberg", "")
    text = text.replace("Gutenberg", "")

    # Remove carriage returns
    text = text.replace("\r", "")

    # fix quotes
    text = text.replace("“", "\"")
    text = text.replace("”", "\"")

    # Replace any capital letter at the start of a word with ^ followed by the lowercase letter
    text = re.sub(r"(?<![a-zA-Z])([A-Z])", lambda match: f"^{match.group(0).lower()}", text)

    # Replace all other capital letters with lowercase
    text = re.sub(r"([A-Z])", lambda match: f"{match.group(0).lower()}", text)

    # Remove duplicate whitespace
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r"\t+", "\t", text)

    # Replace whitespace characters with special words
    text = re.sub(r"(\t)", r" zztabzz ", text)
    text = re.sub(r"(\n)", r" zznewlinezz ", text)
    text = re.sub(r"(\s)", r" zzspacezz ", text)

    # Split before and after punctuation
    for punctuation in string.punctuation:
        text = text.replace(punctuation, f" {punctuation} ")

    # Makes sure that any non utf-8 characters don't make it through
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
    return text

In [9]:
def postprocess_text(text):

    # Replace special words with whitespace characters
    text = text.replace("zztabzz", "\t")
    text = text.replace("zznewlinezz", "\n")
    text = text.replace("zzspacezz", " ")

    # Remake capital letters at beginning of words
    text = re.sub(r"\^([a-z])", lambda match: f"{match.group(1).upper()}", text)

    text = text.replace("^", "")

    return text

# def getMyText():
#   path_to_file = tf.keras.utils.get_file('austen.txt', 'https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/austen/austen.txt')

#   text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

#   # path_to_file = tf.keras.utils.get_file('903-0.txt', 'https://www.gutenberg.org/files/903/903-0.txt')
#   # author_text += open(path_to_file, 'rb').read().decode(encoding='utf-8')[2999:-19194]
#   # tf.io.gfile.remove(path_to_file)

#   return preprocess_text(text)

import os
import tensorflow as tf

def getMyText():
    file_name = 'austen.txt'
    file_url = 'https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/austen/austen.txt'
    local_dir = './data/'  # Directory to save the file
    local_path = os.path.join(local_dir, file_name)

    try:
        # Ensure the directory exists
        if not os.path.exists(local_dir):
            os.makedirs(local_dir)

        # Check if the file exists locally
        if os.path.exists(local_path):
            print(f"File '{file_name}' found locally. Using it.")
        else:
            print(f"File '{file_name}' not found locally. Downloading it.")
            # Download the file
            downloaded_path = tf.keras.utils.get_file(file_name, file_url)

            # Save the downloaded file to the designated local directory
            with open(downloaded_path, 'rb') as source_file:
                with open(local_path, 'wb') as dest_file:
                    dest_file.write(source_file.read())

        # Read the file's contents
        with open(local_path, 'rb') as file:
            text = file.read().decode(encoding='utf-8')

        return preprocess_text(text)

    except Exception as e:
        print(f"An error occurred: {e}")
        return None



In [10]:
# Multi Author text import

import os
import tensorflow as tf

# Number of files to save
files = 3

def getMyText(count):
    if(count == 1):
        file_name = 'austen.txt'
        file_url = 'https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/austen/austen.txt'
        local_dir = './data/'  # Directory to save the file
        local_path = os.path.join(local_dir, file_name)

        try:
            # Ensure the directory exists
            if not os.path.exists(local_dir):
                os.makedirs(local_dir)

            # Check if the file exists locally
            if os.path.exists(local_path):
                print(f"File '{file_name}' found locally. Using it.")
            else:
                print(f"File '{file_name}' not found locally. Downloading it.")
                # Download the file
                downloaded_path = tf.keras.utils.get_file(file_name, file_url)

                # Save the downloaded file to the designated local directory
                with open(downloaded_path, 'rb') as source_file:
                    with open(local_path, 'wb') as dest_file:
                        dest_file.write(source_file.read())

            # Read the file's contents
            with open(local_path, 'rb') as file:
                text = file.read().decode(encoding='utf-8')

            return preprocess_text(text)

        except Exception as e:
            print(f"An error occurred: {e}")
            return None
    if(count == 2):
        file_name = 'shakespeare.txt'
        file_url = 'https://www.gutenberg.org/cache/epub/100/pg100.txt'
        local_dir = './data/'  # Directory to save the file
        local_path = os.path.join(local_dir, file_name)

        try:
            # Ensure the directory exists
            if not os.path.exists(local_dir):
                os.makedirs(local_dir)

            # Check if the file exists locally
            if os.path.exists(local_path):
                print(f"File '{file_name}' found locally. Using it.")
            else:
                print(f"File '{file_name}' not found locally. Downloading it.")
                # Download the file
                downloaded_path = tf.keras.utils.get_file(file_name, file_url)

                # Save the downloaded file to the designated local directory
                with open(downloaded_path, 'rb') as source_file:
                    with open(local_path, 'wb') as dest_file:
                        dest_file.write(source_file.read())

            # Read the file's contents
            with open(local_path, 'rb') as file:
                text = file.read().decode(encoding='utf-8')

            return preprocess_text(text)

        except Exception as e:
            print(f"An error occurred: {e}")
            return None
    if(count == 3):
        file_name = 'marktwain.txt'
        file_url = 'raw.githubusercontent.com/Allena9/CSE-450/main/Rnn/data/marktwain.txt'
        local_dir = './data/'  # Directory to save the file
        local_path = os.path.join(local_dir, file_name)

        try:
            # Ensure the directory exists
            if not os.path.exists(local_dir):
                os.makedirs(local_dir)

            # Check if the file exists locally
            if os.path.exists(local_path):
                print(f"File '{file_name}' found locally. Using it.")
            else:
                print(f"File '{file_name}' not found locally. Downloading it.")
                # Download the file
                downloaded_path = tf.keras.utils.get_file(file_name, file_url)

                # Save the downloaded file to the designated local directory
                with open(downloaded_path, 'rb') as source_file:
                    with open(local_path, 'wb') as dest_file:
                        dest_file.write(source_file.read())

            # Read the file's contents
            with open(local_path, 'rb') as file:
                text = file.read().decode(encoding='utf-8')

            return preprocess_text(text)

        except Exception as e:
            print(f"An error occurred: {e}")
            return None

getMyText()

In [11]:
def getRandomText(numbooks = 1, verbose=False):
  # Create a buffer to capture download output messages
  download_log = io.StringIO()
  # Initialize empty string to store text from books
  text_random = ''
  # Loop through the requested number of books
  for b in range(numbooks):
    # Flag to track whether we've found a suitable book
    foundbook = False
    # Keep trying until we find a valid book
    while(foundbook == False):
      # Generate random book ID between 100-60000
      booknum = random.randint(100,60000)
      if verbose:
        print('Trying Book #: ',booknum)
      # Randomly choose between two URL formats for Project Gutenberg
      if random.random() > 0.5:
        url = 'https://www.gutenberg.org/files/' + str(booknum) + '/' + str(booknum) + '-0.txt'
        filename_temp = str(booknum) + '-0.txt'
      else:
        url = 'https://www.gutenberg.org/cache/epub/' + str(booknum) + '/pg' + str(booknum) + '.txt'
        filename_temp = 'pg' + str(booknum) + '.txt'
      if verbose:
        print('Trying: ', url)
      try:
        # Download the file, either showing progress or hiding it based on verbose flag
        if verbose:
          path_to_file_temp = tf.keras.utils.get_file(filename_temp, url)
        else:
          with contextlib.redirect_stdout(download_log):
            path_to_file_temp = tf.keras.utils.get_file(filename_temp, url)
        # Read the downloaded file and decode as UTF-8
        temptext = open(path_to_file_temp, 'rb').read().decode(encoding='utf-8')
        # Delete the temporary file after reading its contents
        tf.io.gfile.remove(path_to_file_temp)
        # Check if the book is in English
        if (temptext.find('Language: English') >= 0):
          # Add slight randomness to the starting position
          offset = random.randint(-20,20)
          # Skip the header/preamble text (typically first 2000 chars)
          header = 2000
          # Target length for extracted text
          total_length = 200000
          # Amount to trim from the end of the book
          chopoffend = 10000
          # For long books: extract a fixed-size chunk
          if len(temptext) > (header+total_length+offset+chopoffend):
            foundbook = True
            text_random += temptext[header+offset:header+total_length+offset]
            #print("Yes: " + str(booknum))
            if verbose:
              print('New size of dataset: ', len(text_random))
          # For medium-length books: take most of the book minus header and footer
          elif len(temptext) > (header+12000):
            foundbook = True
            text_random += temptext[header:-chopoffend]
            #print("Yes (smaller): " + str(booknum))
            if verbose:
              print('New size of dataset: ', len(text_random))
          # Skip books that are too short
          else:
            if verbose:
              print('Not long enough. Trying again...')
            #print("No: " + str(booknum) + " too short")
        # Skip non-English books
        else:
          if verbose:
            print('Not English. Trying again...')
          #print("No: " + str(booknum) + " not English")
        # Clean up memory
        del temptext
      # Handle any exceptions during download/processing
      except:
        if verbose:
          print('Not valid file. Trying again...')
        #print("No: " + str(booknum) + " not valid")
        foundbook = False
    if verbose:
      print("Found " + str(b+1) + " books so far...")
  # Clean up memory
  del download_log
  # The following commented code appears to be for further processing that was replaced
  #text_random = "".join(c for c in text_random if c in vocab)
  #all_ids_random = ids_from_chars(tf.strings.unicode_split(text_random, 'UTF-8'))
  #ids_dataset_random = tf.data.Dataset.from_tensor_slices(all_ids_random)
  #sequences_random = ids_dataset_random.batch(seq_length+1, drop_remainder=True)
  #dataset_random = sequences_random.map(split_input_target)
  #dataset_random = (dataset_random.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))
  #return dataset_random
  
  # Pass the collected text to a preprocessing function and return the result
  return preprocess_text(text_random)

In [12]:
if restart:
  vocab_text = getMyText(files)

File 'marktwain.txt' found locally. Using it.


Make vocabulary (Adapted from TensorFlow word embedding tutorial)

---



In [13]:
# Vocabulary size and number of words in a sequence.
vocab_size = 12288
sequence_length = 128

In [14]:
def custom_standardization(input_string):
    # Lowercase the text
    lowercase = tf.strings.lower(input_string)
    
    # Handle common punctuation marks one by one
    # This avoids regex escaping issues
    lowercase = tf.strings.regex_replace(lowercase, r'\.', ' . ')  # Period
    lowercase = tf.strings.regex_replace(lowercase, r',', ' , ')   # Comma
    lowercase = tf.strings.regex_replace(lowercase, r'!', ' ! ')   # Exclamation
    lowercase = tf.strings.regex_replace(lowercase, r'\?', ' ? ')  # Question mark
    lowercase = tf.strings.regex_replace(lowercase, r';', ' ; ')   # Semicolon
    lowercase = tf.strings.regex_replace(lowercase, r':', ' : ')   # Colon
    lowercase = tf.strings.regex_replace(lowercase, r'\(', ' ( ')  # Open parenthesis
    lowercase = tf.strings.regex_replace(lowercase, r'\)', ' ) ')  # Close parenthesis
    lowercase = tf.strings.regex_replace(lowercase, r'\[', ' [ ')  # Open bracket
    lowercase = tf.strings.regex_replace(lowercase, r'\]', ' ] ')  # Close bracket
    lowercase = tf.strings.regex_replace(lowercase, r'\{', ' { ')  # Open brace
    lowercase = tf.strings.regex_replace(lowercase, r'\}', ' } ')  # Close brace
    lowercase = tf.strings.regex_replace(lowercase, r'"', ' " ')   # Double quote
    lowercase = tf.strings.regex_replace(lowercase, r"'", " ' ")   # Single quote
    lowercase = tf.strings.regex_replace(lowercase, r'-', ' - ')   # Hyphen
    lowercase = tf.strings.regex_replace(lowercase, r'\*', '')   # Hyphen
    
    # Remove extra whitespace
    return tf.strings.regex_replace(lowercase, r'\s+', ' ')

if restart:
  # Use the text vectorization layer to normalize, split, and map strings to
  # integers. Note that the layer uses the custom standardization defined above.
  # Set maximum_sequence length as all samples are not of the same length.
  vectorize_layer = TextVectorization(
      standardize=custom_standardization,
      split='whitespace',
      max_tokens=vocab_size,
      output_mode='int',
      #output_sequence_length=sequence_length
      )

In [15]:
if restart:
  # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
  vectorize_layer.adapt([vocab_text])

In [16]:
if restart:
  vocabulary = vectorize_layer.get_vocabulary()

Save Vocabulary

In [17]:
if restart:
  with open(path + "vocabulary.txt", "w") as file:
    for word in vocabulary:
        file.write(word + "\n")

Load Saved Vocabulary

In [18]:
if restart == False:
  with open(path + "vocabulary.txt", "r") as file:
      vocabulary = [word.strip() for word in file.readlines()]
      vocabulary = vocabulary

  vectorize_layer = TextVectorization(
      vocabulary=vocabulary,
      standardize='lower',
      split='whitespace',
      max_tokens=vocab_size,
      output_mode='int',
      #output_sequence_length=sequence_length
      )

In [19]:
print(vocabulary[:20])
print(vocabulary[-20:])

['', '[UNK]', 'zzspacezz', '^', ',', 'the', 'and', '.', 'a', 'to', 'of', '-', '"', 'i', 'it', 'was', 'in', 'he', ';', 'that']
['toofor', 'tooevery', 'tooeating', 'tooand', 'tooa', 'tontine', 'toniest', 'tongs', 'tones', 'tomyou', 'tomthey', 'tomthats', 'tomtake', 'tomswhy', 'tomoh', 'tomnow', 'tomnobody', 'tommy', 'tomleast', 'tomit']


Turn text into a dataset

In [20]:
# This function will generate our sequence pairs:
def split_input_target(sequence):
    input_ids = sequence[:-1]
    target_ids = sequence[1:]
    return input_ids, target_ids

# This function will create the dataset
def text_to_dataset(text):
  all_ids = vectorize_layer(text)
  ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
  del all_ids
  sequences = ids_dataset.batch(sequence_length+1, drop_remainder=True)
  del ids_dataset

  # Call the function for every sequence in our list to create a new dataset
  # of input->target pairs
  dataset = sequences.map(split_input_target)
  del sequences

  # shuffle


  return dataset

Test on vocab text

In [21]:
if restart:
  vocab_ds = text_to_dataset(vocab_text)

In [22]:
def text_from_ids(ids):
  text = ''.join([vocabulary[index] for index in ids])
  return postprocess_text(text)

vocabulary_adjusted = vocabulary
vocabulary_adjusted[0] = '[UNK]'
vocabulary_adjusted[1] = ''

words_from_ids = StringLookup(vocabulary=vocabulary_adjusted, invert=True)

In [23]:
if restart:
  for input_example, target_example in vocab_ds.take(1):
    print("Input: ")
    print(input_example)
    print(text_from_ids(input_example))
    print(words_from_ids(input_example))
    print("Target: ")
    print(target_example)
    print(text_from_ids(target_example))

Input: 
tf.Tensor(
[    3   257     2     3    13     7     2     3     5     2  2859     2
    10     2     5     2     3   311     2     6     2     5     2     3
  2575     7     2     3    16     2     5     2  1063     2   433     2
    10     2     3   759     4     2    27     2     8     2   620     2
 11069     2   122     2    16     2     5     2   594     2   624     2
    10     2     5     2  7198     2  1674     4     2     8     2   165
     2    15     2   723     2     9     2     8     2   195     2   449
     2    10     2     5     2   260     2    10     2     3   517     4
     2   120     2    99     2    44     2   189     2    29     7     2
     3    27     2     5     2   198     2   122     2   164     2     3
   962     2   443     2    15     2   723     2], shape=(128,), dtype=int64)
Chapter I. The birth of the Prince and the Pauper. In the ancient city of London, on a certain autumn day in the second quarter of the sixteenth century, a boy was born to a

In [24]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

def setup_dataset(dataset):
  dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
  return dataset


In [25]:
if restart:
  vocab_ds = setup_dataset(vocab_ds)

## III. Build the model

Next, we'll build our model. Up until this point, you've been using the Keras symbolic, or imperative API for creating your models. Doing something like:

    model = tf.keras.models.Sequentla()
    model.add(tf.keras.layers.Dense(80, activation='relu))
    etc...

However, tensorflow has another way to build models called the Functional API, which gives us a lot more control over what happens inside the model. You can read more about [the differences and when to use each here](https://blog.tensorflow.org/2019/01/what-are-symbolic-and-imperative-apis.html).

We'll use the functional API for our RNN in this example. This will involve defining our model as a custom subclass of `tf.keras.Model`.

If you're not familiar with classes in python, you might want to review [this quick tutorial](https://www.w3schools.com/python/python_classes.asp), as well as [this one on class inheritance](https://www.w3schools.com/python/python_inheritance.asp).

Using a functional model is important for our situation because we're not just training it to predict a single character for a single sequence, but as we make predictions with it, we need it to remember those predictions as use that memory as it makes new predictions.


In [26]:
# Create our custom model. Given a sequence of characters, this
# model's job is to predict what character should come next.
class AustenTextModel(tf.keras.Model):

  # This is our class constructor method, it will be executed when
  # we first create an instance of the class
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()

    # Our model will have three layers:

    # 1. An embedding layer that handles the encoding of our vocabulary into
    #    a vector of values suitable for a neural network
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    # 2. A GRU layer that handles the "memory" aspects of our RNN. If you're
    #    wondering why we use GRU instead of LSTM, and whether LSTM is better,
    #    take a look at this article: https://datascience.stackexchange.com/questions/14581/when-to-use-gru-over-lstm
    #    then consider trying out LSTM instead (or in addition to!)
    #self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
    self.lstm1 = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)
    self.lstm2 = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)
    self.lstm3 = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)
    #self.lstm4 = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)


    self.hidden1 = tf.keras.layers.Dense(embedding_dim*64, activation='relu')
    self.hidden2 = tf.keras.layers.Dense(embedding_dim*16, activation='relu')
    #self.hidden3 = tf.keras.layers.Dense(embedding_dim*4, activation='relu')

    # 3. Our output layer that will give us a set of probabilities for each
    #    character in our vocabulary.
    self.dense = tf.keras.layers.Dense(vocab_size)

  # This function will be executed for each epoch of our training. Here
  # we will manually feed information from one layer of our network to the
  # next.
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs

    # 1. Feed the inputs into the embedding layer, and tell it if we are
    #    training or predicting
    x = self.embedding(x, training=training)

    # 2. If we don't have any state in memory yet, get the initial random state
    #    from our GRUI layer.
    batch_size = tf.shape(inputs)[0]

    if states is None:
      states1 = [tf.zeros([batch_size, self.lstm1.units]), tf.zeros([batch_size, self.lstm1.units])]
      states2 = [tf.zeros([batch_size, self.lstm2.units]), tf.zeros([batch_size, self.lstm2.units])]
      states3 = [tf.zeros([batch_size, self.lstm3.units]), tf.zeros([batch_size, self.lstm3.units])]
      #states4 = [tf.zeros([batch_size, self.lstm4.units]), tf.zeros([batch_size, self.lstm4.units])]
    else:
      states1 = states[0]
      states2 = states[1]
      states3 = states[2]
      #states4 = states[3]
    # 3. Now, feed the vectorized input along with the current state of memory
    #    into the gru layer.
    x, state_h_1, state_c_1 = self.lstm1(x, initial_state=states1, training=training)
    states_out_1 = [state_h_1,state_c_1]

    x, state_h_2, state_c_2 = self.lstm2(x, initial_state=states2, training=training)
    states_out_2 = [state_h_2,state_c_2]

    x, state_h_3, state_c_3 = self.lstm3(x, initial_state=states3, training=training)
    states_out_3 = [state_h_3,state_c_3]

    #x, state_h_4, state_c_4 = self.lstm4(x, initial_state=states4, training=training)
    #states_out_4 = [state_h_4,state_c_4]

    states_out = [states_out_1, states_out_2, states_out_3]#, states_out_4]
    #states_out = [states_out_1, states_out_2]

    x = self.hidden1(x,training=training)
    x = self.hidden2(x,training=training)
    #x = self.hidden3(x,training=training)
    # 4. Finally, pass the results on to the dense layer
    x = self.dense(x, training=training)

    # 5. Return the results
    if return_state:
      return x, states_out
    else:
      return x

In [27]:
if restart:
  dataset = vocab_ds
  del vocab_text
  del vocab_ds
else:
  new_text = getRandomText(numbooks = 10)
  dataset = text_to_dataset(new_text)
  del new_text
  dataset = setup_dataset(dataset)

In [28]:
# Create an instance of our model
#vocab_size=len(ids_from_chars.get_vocabulary())
embedding_dim = 128
rnn_units = 512

model = AustenTextModel(vocab_size, embedding_dim, rnn_units)

In [29]:
# Verify the output of our model is correct by running one sample through
# This will also compile the model for us. This step will take a bit.
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 128, 12288) # (batch_size, sequence_length, vocab_size)


In [30]:
# Now let's view the model summary
model.summary()

Model: "austen_text_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1572864   
_________________________________________________________________
lstm (LSTM)                  multiple                  1312768   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  2099200   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  2099200   
_________________________________________________________________
dense (Dense)                multiple                  4202496   
_________________________________________________________________
dense_1 (Dense)              multiple                  16779264  
_________________________________________________________________
dense_2 (Dense)              multiple            

In [31]:
# Here's the code we'll use to sample for us. It has some extra steps to apply
# the temperature to the distribution, and to make sure we don't get empty
# characters in our text. Most importantly, it will keep track of our model
# state for us.

class OneStep(tf.keras.Model):
  def __init__(self, model, vectorize_layer, vocabulary, temperature=1):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.vectorize_layer = vectorize_layer
    self.vocabulary = vocabulary
    #print("initialized")

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = StringLookup(vocabulary=list(vocabulary))(['', '[UNK]'])[:, None]
    #print(skip_ids)
    #print("3")
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(vocabulary)])
    #print("4")
    self.prediction_mask = tf.sparse.to_dense(sparse_mask,validate_indices=False)
    #print("5")

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    #input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.vectorize_layer(inputs)
    #print(input_ids)

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states =  self.model(inputs=input_ids, states=states,
                                          return_state=True)
    del input_ids
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    del predicted_logits
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    #print(predicted_ids[0])

    # Return the characters and model state.
    return words_from_ids(predicted_ids), states


In [32]:
def produce_sample(model, vectorize_layer, vocabulary, temp, epoch, prompt):
  # Create an instance of the character generator
  #print("entered")
  one_step_model = OneStep(model, vectorize_layer, vocabulary, temp)
  #print("rand one step")
  # Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
  # as its starting text
  states = None
  next_char = tf.constant([preprocess_text(prompt)])
  result = [tf.constant([prompt])]

  for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    #print(next_char)
    result.append(next_char)
    #print(result)

  result = tf.strings.join(result)
  #print(result)

  # Print the results formatted.
  #print('Temp: ' + str(temp) + '\n')
  print(postprocess_text(result[0].numpy().decode('utf-8')))
  #print('\n\n')
  print('Epoch: ' + str(epoch) + '\n', file=open(path + 'tree.txt', 'a'))
  print('Temp: ' + str(temp) + '\n', file=open(path + 'tree.txt', 'a'))
  print(postprocess_text(result[0].numpy().decode('utf-8')), file=open(path + 'tree.txt', 'a'))
  print('\n\n', file=open(path + 'tree.txt', 'a'))
  del states
  del next_char
  del result

## IV. Train the model

For our purposes, we'll be using [categorical cross entropy](https://machinelearningmastery.com/cross-entropy-for-machine-learning/) as our loss function*. Also, our model will be outputting ["logits" rather than normalized probabilities](https://stackoverflow.com/questions/41455101/what-is-the-meaning-of-the-word-logits-in-tensorflow), because we'll be doing further transformations on the output later.


\* Note that since our model deals with integer encoding rather than one-hot encoding, we'll specifically be using [sparse categorical cross entropy](https://stats.stackexchange.com/questions/326065/cross-entropy-vs-sparse-cross-entropy-when-to-use-one-over-the-other).

In [33]:
sherlock_text = getMyText(files)

File 'marktwain.txt' found locally. Using it.


if restart == False:
  model.load_weights(path + "lstm_gru_SH_modelweights_fall2023-random_urls.h5")

In [34]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.Adam(learning_rate=0.002)
model.compile(optimizer=opt, loss=loss)

epoch_offset = 10
num_epochs_total = 30
if restart:
  start_epoch = 0
else:
  start_epoch = epoch_to_pickup
for e in range(start_epoch, num_epochs_total):
  success = False
  while(success == False):
    try:
      print("epoch: ", e)
      if e < num_epochs_total-epoch_offset:
        new_text = getRandomText(numbooks = 20)
      else:
        new_text = sherlock_text + getRandomText(numbooks = (num_epochs_total - e)//10)
      # new_text = getMyText(files)
      dataset = text_to_dataset(new_text)
      del new_text
      dataset = setup_dataset(dataset)
      if(e > num_epochs_total-epoch_offset):
        #opt = tf.keras.optimizers.Adam(learning_rate=0.002*(0.97**e))
        #model.compile(optimizer=opt, loss=loss)
        model.optimizer.learning_rate.assign(0.002*(0.99**e))
      model.fit(dataset, epochs=1, verbose=1)
      print("finished training...")
      del dataset
      #print("saving weights...")
      #model.save_weights(path + "lstm_gru_SH_modelweights_fall2023-random_urls.h5")
      #print("weights saved...")
      if(e > num_epochs_total-epoch_offset):
        for temp in [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
          produce_sample(model,vectorize_layer,vocabulary, temp, e, 'The world seemed like such a peaceful place until the magic tree was discovered in London.')
        print("samples produced...")
        gc.collect()
        print("garbage collected...")
        tf.keras.backend.clear_session()
        print("session cleared (to save memory)...")
        #tf.config.experimental.reset_all()
      success = True
    except:
      gc.collect()
      tf.keras.backend.clear_session()
      #tf.config.experimental.reset_all()
      try:
        del dataset
      except:
        print("dataset already deleted")
      print("retrying epoch: " , e)

# Send training complete notification 
notification.notify(
    title='Training Complete!',
    message = 'Done!',
    app_icon=None,
    timeout=10,
)

epoch:  0
178/178 [==============================] - 22s 108ms/step - loss: 3.7308
finished training...
epoch:  1
161/161 [==============================] - 18s 108ms/step - loss: 2.7343
finished training...
epoch:  2
146/146 [==============================] - 16s 108ms/step - loss: 2.5627
finished training...
epoch:  3
165/165 [==============================] - 19s 108ms/step - loss: 2.5426
finished training...
epoch:  4
174/174 [==============================] - 20s 109ms/step - loss: 2.4797
finished training...
epoch:  5
150/150 [==============================] - 17s 109ms/step - loss: 2.4013
finished training...
epoch:  6
179/179 [==============================] - 20s 109ms/step - loss: 2.3725
finished training...
epoch:  7
150/150 [==============================] - 17s 107ms/step - loss: 2.3412
finished training...
epoch:  8
158/158 [==============================] - 18s 109ms/step - loss: 2.3148
finished training...
epoch:  9
159/159 [==============================] - 18s 107ms/s